https://stackoverflow.com/questions/59736682/find-nearest-location-coordinates-in-land-using-python

In [1]:
import numpy as np
import pandas as pd
import geopandas as gp
import geopy
from geopy.geocoders import Nominatim
import time

In [2]:
df = pd.read_csv('./datasets/Anti-Shipping_Activity_Messages.csv')
df.dropna(inplace = True)

In [3]:
df['Y'] = [str(x) for x in df['Y']]
df['X'] = [str(x) for x in df['X']]

In [4]:
df['coords'] = df['Y'] + ',' + df['X']

In [5]:
gloc = Nominatim(user_agent = 'Anything')

In [6]:
def closest_country(c):
    time.sleep(1)
    try:
        c = gloc.reverse(c, language = 'en')
        return c.raw['address']['country']
    except:
        return 'International Waters'

#https://www.geeksforgeeks.org/get-the-city-state-and-country-names-from-latitude-and-longitude-using-python/

In [7]:
# df_locations_1_250 = df['coords'][:250].apply(lambda x: closest_country(x))
# df_locations_250_500 = df['coords'][250:500].apply(lambda x: closest_country(x))
# df_locations_500_750 = df['coords'][500:750].apply(lambda x: closest_country(x))
# df_locations_750_1000 = df['coords'][750:1000].apply(lambda x: closest_country(x))
# df_locations_1000_1500 = df['coords'][1000:1500].apply(lambda x: closest_country(x))
# df_locations_1500_2500 = df['coords'][1500:2500].apply(lambda x: closest_country(x))
# df_locations_2500_end = df['coords'][2500:].apply(lambda x: closest_country(x))
# df.to_csv('./datasets/esri_w_country_columns')

In [8]:
df = pd.read_csv('./datasets/esri_w_country_columns')

In [ ]:
# poverty = pd.read_csv('./sdg_piracy_sheets/csv_data/t111.csv', index_col =[0]) 
# land_rights = pd.read_csv('./sdg_piracy_sheets/csv_data/t142.csv', index_col =[0]) 
# education = pd.read_csv('./sdg_piracy_sheets/csv_data/t411.csv', index_col =[0]) 
# overfishing = pd.read_csv('./sdg_piracy_sheets/csv_data/t1441.csv', index_col =[0]) 
# sea_law = pd.read_csv('./sdg_piracy_sheets/csv_data/t14c1.csv', index_col =[0]) 

In [ ]:
# land_rights.SeriesDescription.unique()

In [ ]:
# Value based on year/country (keep Series Description)
# education.GeoAreaName.value_counts()

In [ ]:
# overfishing.Value

In [ ]:
# sea_law.SeriesDescription.unique()

In [ ]:
# sea_law.shape

In [9]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [10]:
#Breaking into month, year, day and time.
df['year'] = [x[0:4] for x in df.dateofocc]
df['month'] = [x[5:7] for x in df.dateofocc]
df['day'] = [x[8:10] for x in df.dateofocc]
df['time'] = [x[11:] for x in df.dateofocc]

In [11]:
df[df['time'] == '00:00:00+00'].count()
#most rows are missing 'time'

X                  7792
Y                  7792
OBJECTID           7792
reference          7792
dateofocc          7792
subreg             7792
hostility_d        7792
victim_d           7792
description        7792
hostilitytype_l    7792
victim_l           7792
navarea            7792
coords             7792
country            7792
year               7792
month              7792
day                7792
time               7792
dtype: int64

In [12]:
#Dropping time column
df.drop(columns = 'time', inplace = True)

In [13]:
#Dropping original date of occurence column
df.drop(columns = 'dateofocc', inplace = True)

In [14]:
#Dropping 'OBJECTID'
df.drop(columns = 'OBJECTID', axis = 1, inplace = True)

In [15]:
#Lowercase all text
for i in df.columns:
    if df[i].dtypes == 'O':
        df[i] = df[i].str.lower()

In [16]:
df.hostilitytype_l.value_counts()

1.0    6971
3.0     537
4.0     163
6.0     124
2.0      47
5.0      40
9.0       6
7.0       1
Name: hostilitytype_l, dtype: int64

In [17]:
host_lst = ['pirate_assaults', 
            'navel_engagement', 
            'suspicious_approach', 
            'kidnapping', 
            'unknown', 
            'other', 
            'hijacking', 
            'no entries', 
            'attempted_boarding']

df.hostilitytype_l = [host_lst[int(x) - 1] for x in df.hostilitytype_l]

In [18]:
victim_lst = ['anchored_vessel', 
            'barge', 
            'cargo_ship', 
            'fishing_vessel', 
            'merchant_vessel', 
            'offshore_vessel', 
            'passenger_ship', 
            'sailing_vessel', 
            'tanker',
            'tugboat',
            'vessel',
            'unknown',
            'other']

df.victim_l = [victim_lst[int(x) - 1] for x in df.victim_l]

In [19]:
#Dummy all categorical columns
to_dummy = ['subreg', 'hostilitytype_l', 'victim_l', 'navarea', 'month']

for i in to_dummy:
    df = pd.concat([df, pd.get_dummies(df[i], prefix = i)], axis=1)
#df.drop(columns = to_dummy, axis = 1)

In [20]:
#ADD TO CLEANING

success_dict = {
    'pirate_assaults':1,      
    'suspicious_approach':0,  
    'kidnapping':1,
    'other':0,
    'navel_engagement':0, 
    'unknown':0,
    'attempted_boarding':0,
    'hijacking':1
}



df['pirate_success'] = df.hostilitytype_l.map(success_dict)

In [30]:
df['year'] = df['year'].astype(int)

In [31]:
#Incorporate only observations from 2010 on.
df = df[df['year'] >= 2010]

In [41]:
df.drop(columns = ['reference', 'hostility_d', 'victim_d', 'description'], inplace = True)

In [43]:
df.shape

(3278, 100)

In [45]:
poverty = pd.read_csv('./datasets/poverty.csv')

In [48]:
#BRING TO BOTTOM OF PAGE

# df.to_csv('./datasets/cleaned_pirate_activity_eda.csv')
# df = pd.read_csv('./datasets/cleaned_pirate_activity_eda.csv', index_col=0)
# df.head()

In [47]:
df.head()

,X,Y,subreg,hostilitytype_l,victim_l,navarea,coords,country,year,month,...,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12,pirate_success
32,112.000000,16.500000,93,pirate_assaults,fishing_vessel,xi,"16.500000000429303,112.00000000037",china,2015,7,...,0,0,0,1,0,0,0,0,0,1
33,70.216667,23.016667,63,pirate_assaults,tanker,viii,"23.016666700086898,70.2166667001745",india,2015,7,...,0,0,0,1,0,0,0,0,0,1
34,70.216667,23.016667,63,pirate_assaults,tanker,viii,"23.016666700086898,70.2166667001745",india,2015,7,...,0,0,0,1,0,0,0,0,0,1
35,101.983333,2.050000,71,pirate_assaults,tanker,xi,"2.05000000027673,101.98333330015",indonesia,2015,8,...,0,0,0,0,1,0,0,0,0,1
36,103.566667,1.100000,71,pirate_assaults,cargo_ship,xi,"1.1000000001110901,103.56666670030899",indonesia,2015,8,...,0,0,0,0,1,0,0,0,0,1


In [50]:
countries = ['The Bahamas', 'Indonesia','International Waters', 'Eritrea', 'India', 'Brazil',
 'Somalia', 'Ecuador', 'Philippines', 'Malta', 'China', 'Cameroon', 'Sri Lanka','Nicaragua',
 'Senegal', 'Bangladesh', 'Vietnam', 'Malaysia', 'Mozambique', 'Guyana', 'Algeria', 'Tanzania',
 'Lebanon', 'Visayas', 'Colombia', 'Nigeria', 'Egypt', 'Thailand', 'Russia', 'Guinea',
 'Morocco', "Côte d'Ivoire", 'Portugal', 'Japan', 'Myanmar', 'Mindanao',
 'Dominican Republic', 'Iran', 'Venezuela', 'Ghana', 'Angola', 'Sierra Leone', 
 'Democratic Republic of the Congo', 'Madagascar', 'Turkey', 'Peru', 'Italy', 'Oman', 'Djibouti',
 'North Korea', 'Greece', 'Yemen', 'Taiwan', 'Comoros', 'Papua New Guinea', 'Jamaica', 'Saudi Arabia',
 'Netherlands', 'Panama', 'Singapore', 'Kenya', 'France', 'Pakistan', 'United States', 'Gabon',
 'Congo-Brazzaville', 'Belgium', 'Brunei', 'Cyprus', 'Haiti', 'Liberia', 'Belize', 'Qatar', 'Solomon Islands',
 'Equatorial Guinea', 'Guatemala', 'Fiji', 'South Africa', 'Tunisia', 'Mauritania', 'United Arab Emirates',
 'Germany', 'Mexico', 'Montenegro', 'Togo', 'Honduras', 'United Kingdom', 'Benin', 'Trinidad and Tobago',
 'Bulgaria', 'Georgia', 'Cuba', 'Iraq', 'Suriname', 'Australia', 'El Salvador', 'Romania', 'Saint Lucia',
 'Uruguay', 'British Virgin Islands', 'Saint Vincent and the Grenadines', 'Sudan', 'Dominica',
 'Spain', 'Costa Rica', 'Antigua and Barbuda', 'Grenada', 'South Korea', 'Cape Verde', 'Seychelles',
 'Libya', 'Cayman Islands', 'Saint Kitts and Nevis']

countries = [x.lower() for x in countries]

In [53]:
lis = [44, 360, 1, 232, 356, 76, 706, 218, 608, 470, 156, 120, 144, 558, 686, 50, 704, 458, 508, 328, 
          12, 834, 422, 608, 170, 566, 818, 764, 643, 324, 504, 384, 620, 392, 104, 608, 214, 364, 862, 
          288, 24, 694, 180, 450, 792, 604, 380, 512, 262, 408, 300, 887, 156, 174, 598, 388, 682, 528,
         591, 702, 404, 250, 586, 840, 266, 178, 56, 96, 196, 332, 430, 84, 634, 90, 226, 320, 242, 710, 
         788, 478, 784, 276, 484, 499, 768, 340, 826, 204, 780, 100, 268, 192, 368, 740, 36, 222, 642, 
         662, 858, 92, 670, 729, 212, 724, 188, 28, 308, 410, 132, 690, 434, 136, 659]

In [54]:
dic = {countries[i]: lis[i] for i in range(len(countries))}

In [56]:
df['country_code'] = df['country'].map(dic)

In [57]:
df['country_code']

32      156
33      356
34      356
35      360
36      360
       ... 
7884    608
7885     24
7886    360
7887    608
7888      1
Name: country_code, Length: 3278, dtype: int64

In [58]:
df.head()

,X,Y,subreg,hostilitytype_l,victim_l,navarea,coords,country,year,month,...,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12,pirate_success,country_code
32,112.000000,16.500000,93,pirate_assaults,fishing_vessel,xi,"16.500000000429303,112.00000000037",china,2015,7,...,0,0,1,0,0,0,0,0,1,156
33,70.216667,23.016667,63,pirate_assaults,tanker,viii,"23.016666700086898,70.2166667001745",india,2015,7,...,0,0,1,0,0,0,0,0,1,356
34,70.216667,23.016667,63,pirate_assaults,tanker,viii,"23.016666700086898,70.2166667001745",india,2015,7,...,0,0,1,0,0,0,0,0,1,356
35,101.983333,2.050000,71,pirate_assaults,tanker,xi,"2.05000000027673,101.98333330015",indonesia,2015,8,...,0,0,0,1,0,0,0,0,1,360
36,103.566667,1.100000,71,pirate_assaults,cargo_ship,xi,"1.1000000001110901,103.56666670030899",indonesia,2015,8,...,0,0,0,1,0,0,0,0,1,360


In [63]:
poverty

,Unnamed: 0,TimePeriod,Indicator,GeoAreaCode,GeoAreaName,poverty_percentage
0,10,2010,1.1.1,1.0,World,16.0
1,11,2011,1.1.1,1.0,World,13.9
2,12,2012,1.1.1,1.0,World,12.9
3,13,2013,1.1.1,1.0,World,11.4
4,14,2014,1.1.1,1.0,World,10.7
...,...,...,...,...,...,...
2408,4483,2017,1.1.1,894.0,Zambia,58.7
2409,4484,2018,1.1.1,894.0,Zambia,58.7
2410,4485,2019,1.1.1,894.0,Zambia,58.7
2411,4486,2020,1.1.1,894.0,Zambia,58.7


In [68]:
poverty.rename(columns = {'GeoAreaCode':'country_code', 'TimePeriod': 'year'}, inplace = True)

In [72]:
#df1['pricesMatch?'] = np.where(df1['Price1'] == df2['Price2'], 'True', 'False')

df['poverty'] = np.where(poverty['country_code'] == df['country_code'] and df['year'] == poverty['year'], poverty['poverty_percentage'])

ValueError: Can only compare identically-labeled Series objects